### IMC-2025 Rank-5 Solution Notebook
#### Author: Sayan Paul
#### [Link to Kaggle Discussion](https://www.kaggle.com/competitions/image-matching-challenge-2025/discussion/583711)

In [1]:
%cd /kaggle/working/
!rm -r /kaggle/working/mast3r/
!rm -r /kaggle/working/miniconda/
!rm -r /kaggle/working/result/
!rm /kaggle/working/submission.csv

!tar -xzf /kaggle/usr/lib/imc25_mast3r_sfm_install_utility_script_4/mast3r.tar.gz -C /kaggle/working/
!tar -xzf /kaggle/usr/lib/imc25_mast3r_sfm_install_utility_script_4/miniconda.tar.gz -C /kaggle/working/
!ls -lh .

/kaggle/working
rm: cannot remove '/kaggle/working/mast3r/': No such file or directory
rm: cannot remove '/kaggle/working/miniconda/': No such file or directory
rm: cannot remove '/kaggle/working/result/': No such file or directory
rm: cannot remove '/kaggle/working/submission.csv': No such file or directory
total 64K
drwxr-xr-x 10 root root 4.0K Jun 15 10:30 mast3r
drwxr-xr-x 19 root root 4.0K Jun 15 10:18 miniconda
----------  1 root root  55K Jun 17 18:17 __notebook__.ipynb


In [2]:
PREFIX_KAGGLE_ROOT = ""
SUBMISSION_FILE = f"{PREFIX_KAGGLE_ROOT}/kaggle/working/submission.csv"

In [3]:
%cd {PREFIX_KAGGLE_ROOT}/kaggle/working/mast3r

/kaggle/working/mast3r


In [4]:
%%writefile mast3r/demo_colmapdb.py

import os
import copy
import torch

from kapture.converter.colmap.database_extra import kapture_to_colmap
from kapture.converter.colmap.database import COLMAPDatabase

from mast3r.colmap.mapping import kapture_import_image_folder_or_list, run_mast3r_matching
from mast3r.retrieval.processor import Retriever
from mast3r.image_pairs import make_pairs

import mast3r.utils.path_to_dust3r  # noqa
from dust3r.utils.image import load_images


def make_pairs_from_pair_idxs_list(imgs, pair_idxs_list, symmetrize=True):
    pairs = []
    for (i,j) in pair_idxs_list:
        pairs.append((imgs[i], imgs[j]))

    if symmetrize:
        pairs += [(img2, img1) for img1, img2 in pairs]

    return pairs


def get_colmapdb_from_mast3r_matches(outdir, filelist, image_size, model, retrieval_model, 
                                     scenegraph_type, winsize, refid, win_cyclic, shared_intrinsics,
                                     silent, device, delete_cache, image_square_ok, pair_idxs_list=None):

    imgs = load_images(filelist, size=image_size, verbose=not silent, square_ok=image_square_ok)
    if len(imgs) == 1:
        imgs = [imgs[0], copy.deepcopy(imgs[0])]
        imgs[1]['idx'] = 1
        filelist = [filelist[0], filelist[0]]

    scene_graph_params = [scenegraph_type]
    if scenegraph_type in ["swin", "logwin"]:
        scene_graph_params.append(str(winsize))
    elif scenegraph_type == "oneref":
        scene_graph_params.append(str(refid))
    elif scenegraph_type == "retrieval":
        scene_graph_params.append(str(winsize))  # Na
        scene_graph_params.append(str(refid))  # k

    if scenegraph_type in ["swin", "logwin"] and not win_cyclic:
        scene_graph_params.append('noncyclic')
    scene_graph = '-'.join(scene_graph_params)

    sim_matrix = None
    if 'retrieval' in scenegraph_type:
        assert retrieval_model is not None
        retriever = Retriever(retrieval_model, backbone=model, device=device)
        with torch.no_grad():
            sim_matrix = retriever(filelist)

        # Cleanup
        del retriever
        torch.cuda.empty_cache()

    if scene_graph == "pair_idxs_list":
        assert pair_idxs_list is not None
        pairs = make_pairs_from_pair_idxs_list(imgs, pair_idxs_list, symmetrize=True)
    else:
        pairs = make_pairs(imgs, scene_graph=scene_graph, prefilter=None, symmetrize=True, sim_mat=sim_matrix)


    root_path = os.path.commonpath(filelist)
    filelist_relpath = [
        os.path.relpath(filename, root_path).replace('\\', '/')
        for filename in filelist
    ]
    kdata = kapture_import_image_folder_or_list((root_path, filelist_relpath), shared_intrinsics)
    image_pairs = [
        (filelist_relpath[img1['idx']], filelist_relpath[img2['idx']])
        for img1, img2 in pairs
    ]

    colmap_db_path = os.path.join(outdir, 'colmap.db')
    if os.path.isfile(colmap_db_path):
        os.remove(colmap_db_path)

    os.makedirs(os.path.dirname(colmap_db_path), exist_ok=True)
    colmap_db = COLMAPDatabase.connect(colmap_db_path)
    try:
        kapture_to_colmap(kdata, root_path, tar_handler=None, database=colmap_db,
                          keypoints_type=None, descriptors_type=None, export_two_view_geometry=False)
        colmap_image_pairs = run_mast3r_matching(model, image_size, 16, device,
                                                 kdata, root_path, image_pairs, colmap_db,
                                                 False, 5, 1.001,
                                                 False, 3)
        colmap_db.close()
    except Exception as e:
        print(f'Error {e}')
        colmap_db.close()
        exit(1)

    if len(colmap_image_pairs) == 0:
        raise Exception("no matches were kept")

    
    colmap_world_to_cam = {}
    
    pairs_path = os.path.join(outdir, "pairs.txt")
    
    f = open(pairs_path, "w")
    for image_path1, image_path2 in colmap_image_pairs:
        f.write("{} {}\n".format(image_path1, image_path2))
    f.close()
    
    return colmap_db_path, pairs_path



Writing mast3r/demo_colmapdb.py


In [5]:
%%writefile retrieval_processor.py

import os
import argparse
import numpy as np
import torch

import cv2
from glob import glob
from typing import Optional, List, Tuple

from mast3r.model import AsymmetricMASt3R
from mast3r.retrieval.model import RetrievalModel, extract_local_features

try:
    import faiss
    faiss.StandardGpuResources()  # when loading the checkpoint, it will try to instanciate FaissGpuL2Index
except AttributeError as e:
    import asmk.index

    class FaissCpuL2Index(asmk.index.FaissL2Index):
        def __init__(self, gpu_id):
            super().__init__()
            self.gpu_id = gpu_id

        def _faiss_index_flat(self, dim):
            """Return initialized faiss.IndexFlatL2"""
            return faiss.IndexFlatL2(dim)

    asmk.index.FaissGpuL2Index = FaissCpuL2Index

from asmk import asmk_method  # noqa


def get_args_parser():
    parser = argparse.ArgumentParser('Retrieval scores from a set of retrieval', add_help=True, allow_abbrev=False)
    parser.add_argument('--retrieval_model', type=str, default="./checkpoints/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric_retrieval_trainingfree.pth",
                        help="shortname of a retrieval model or path to the corresponding .pth")
    parser.add_argument('--backbone_model', type=str, default="./checkpoints/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric.pth")

    parser.add_argument('--input_dir', type=str, required=True,
                        help="directory containing images")
    parser.add_argument('--out_dir', type=str, required=True, help="numpy file where to store the matrix score")
    return parser


def get_impaths(imlistfile):
    with open(imlistfile, 'r') as fid:
        impaths = [f for f in imlistfile.read().splitlines() if not f.startswith('#')
                   and len(f) > 0]  # ignore comments and empty lines
    return impaths


def get_impaths_from_imdir(imdir, extensions=['png', 'jpg', 'PNG', 'JPG']):
    assert os.path.isdir(imdir)
    impaths = [os.path.join(imdir, f) for f in sorted(os.listdir(imdir)) if any(f.endswith(ext) for ext in extensions)]
    return impaths


def get_impaths_from_imdir_or_imlistfile(input_imdir_or_imlistfile):
    if os.path.isfile(input_imdir_or_imlistfile):
        return get_impaths(input_imdir_or_imlistfile)
    else:
        return get_impaths_from_imdir(input_imdir_or_imlistfile)


class Retriever(object):
    def __init__(self, modelname, backbone=None, device='cuda'):
        # load the model
        assert os.path.isfile(modelname), modelname
        print(f'Loading retrieval model from {modelname}')
        ckpt = torch.load(modelname, 'cpu')  # TODO from pretrained to download it automatically
        ckpt_args = ckpt['args']
        if backbone is None:
            backbone = AsymmetricMASt3R.from_pretrained(ckpt_args.pretrained)
        self.model = RetrievalModel(
            backbone, freeze_backbone=ckpt_args.freeze_backbone, prewhiten=ckpt_args.prewhiten,
            hdims=list(map(int, ckpt_args.hdims.split('_'))) if len(ckpt_args.hdims) > 0 else "",
            residual=getattr(ckpt_args, 'residual', False), postwhiten=ckpt_args.postwhiten,
            featweights=ckpt_args.featweights, nfeat=ckpt_args.nfeat
        ).to(device)
        self.device = device
        msg = self.model.load_state_dict(ckpt['model'], strict=False)
        assert all(k.startswith('backbone') for k in msg.missing_keys)
        assert len(msg.unexpected_keys) == 0
        self.imsize = ckpt_args.imsize

        # load the asmk codebook
        dname, bname = os.path.split(modelname)  # TODO they should both be in the same file ?
        bname_splits = bname.split('_')
        cache_codebook_fname = os.path.join(dname, '_'.join(bname_splits[:-1]) + '_codebook.pkl')
        assert os.path.isfile(cache_codebook_fname), cache_codebook_fname
        asmk_params = {'index': {'gpu_id': 0}, 'train_codebook': {'codebook': {'size': '64k'}},
                       'build_ivf': {'kernel': {'binary': True}, 'ivf': {'use_idf': False},
                                     'quantize': {'multiple_assignment': 1}, 'aggregate': {}},
                       'query_ivf': {'quantize': {'multiple_assignment': 5}, 'aggregate': {},
                                     'search': {'topk': None},
                                     'similarity': {'similarity_threshold': 0.0, 'alpha': 3.0}}}
        asmk_params['train_codebook']['codebook']['size'] = ckpt_args.nclusters
        self.asmk = asmk_method.ASMKMethod.initialize_untrained(asmk_params)
        self.asmk = self.asmk.train_codebook(None, cache_path=cache_codebook_fname)

    def __call__(self, input_imdir_or_imlistfile, outfile=None):
        # get impaths
        if isinstance(input_imdir_or_imlistfile, str):
            impaths = get_impaths_from_imdir_or_imlistfile(input_imdir_or_imlistfile)
        else:
            impaths = input_imdir_or_imlistfile  # we're assuming a list has been passed
        print(f'Found {len(impaths)} images')

        # build the database
        feat, ids = extract_local_features(self.model, impaths, self.imsize, tocpu=True, device=self.device)
        feat = feat.cpu().numpy()
        ids = ids.cpu().numpy()
        asmk_dataset = self.asmk.build_ivf(feat, ids)

        # we actually retrieve the same set of images
        metadata, query_ids, ranks, ranked_scores = asmk_dataset.query_ivf(feat, ids)

        # well ... scores are actually reordered according to ranks ...
        # so we redo it the other way around...
        scores = np.empty_like(ranked_scores)
        scores[np.arange(ranked_scores.shape[0])[:, None], ranks] = ranked_scores

        # save
        if outfile is not None:
            if not os.path.isdir(os.path.dirname(outfile)):
                os.makedirs(os.path.dirname(outfile), exist_ok=True)
            np.save(outfile, scores)
            print(f'Scores matrix saved in {outfile}')
        return scores


#--------------------------------------------------------------------------------------------------------------------------

def get_similarity_pairs(
    sim_matrix: np.ndarray,
    top_k: Optional[int] = None,
    topk_percentile: Optional[float] = None,
    sim_score_thres: Optional[float] = None,
    include_symmetric: bool = False,
    top_k_max : Optional[int] = 50,
) -> List[Tuple[int, int]]:
    
    n = sim_matrix.shape[0]
    pairs = set()

    for i in range(n):
        sim_list = []
        for j in range(n):
            if i == j:
                continue

            score = sim_matrix[i, j]
            if sim_score_thres is not None and score < sim_score_thres:
                continue

            sim_list.append((j, score))

        sim_list.sort(key=lambda x: -x[1])

        if topk_percentile is not None:
            k = max(1, int(topk_percentile * len(sim_list)))
            # k is a subset of sim_list based on ratio, so can't be greater than it's len, but we can restrict it to top_k_max
            if top_k is None:
                sim_list = sim_list[:min(k, top_k_max)]
            else:
                sim_list = sim_list[:min(max(min(k, top_k_max), top_k), len(sim_list)-1)]
        elif top_k is not None:
            # here top_k can be greater than len of sim_list, so must be restricted
            sim_list = sim_list[:min(top_k, len(sim_list)-1)]

        for j, _ in sim_list:
            if include_symmetric:
                pairs.add((i, j))
            else:
                # Only add (i, j) if sim[i, j] >= sim[j, i] to avoid duplicate symmetric matches
                if sim_matrix[i, j] >= sim_matrix[j, i]:
                    pairs.add((i, j))

    return sorted(pairs)


def get_retriever_model(retrieval_model_weights=None,
                       backbone_model_weights=None,
                       device="cpu"):
    
    if retrieval_model_weights is None:
        retrieval_model_weights=f"{os.path.dirname(__file__)}/checkpoints/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric_retrieval_trainingfree.pth"
        backbone_model_weights=f"{os.path.dirname(__file__)}/checkpoints/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric.pth"

    backbone_model = AsymmetricMASt3R.from_pretrained(backbone_model_weights).to(device)
    retriever = Retriever(retrieval_model_weights, backbone=backbone_model, device=device)
    return retriever




Writing retrieval_processor.py


In [6]:
%%writefile mast3r_colmapdb_module.py

import os

from mast3r.model import AsymmetricMASt3R
from mast3r.demo_colmapdb import get_colmapdb_from_mast3r_matches

def get_colmapdb_from_mast3r_matching(outputdir, filelist, scenegraph_type="complete", pair_idxs_list=None,
                                      shared_intrinsics = False, image_square_ok=False, device="cpu"):

    image_size = 512
    backbone_model_weights=f"{os.path.dirname(__file__)}/checkpoints/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric.pth"
    retrieval_model_weights=f"{os.path.dirname(__file__)}/checkpoints/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric_retrieval_trainingfree.pth"
    
    if pair_idxs_list is not None:
        scenegraph_type = "pair_idxs_list"

    if scenegraph_type in ["complete", "pair_idxs_list"]:
        winsize, refid, win_cyclic = None, None, False
    else:
        raise "Other scene_graph types yet to be implemented !!!"

    model = AsymmetricMASt3R.from_pretrained(backbone_model_weights).to(device)
    silent = False

    colmap_db_path, colmap_pairs_path = get_colmapdb_from_mast3r_matches(outputdir, filelist, image_size, model, retrieval_model_weights, 
                                                                        scenegraph_type, winsize, refid, win_cyclic, shared_intrinsics,
                                                                        silent, device, delete_cache=False, image_square_ok=image_square_ok,
                                                                        pair_idxs_list=pair_idxs_list)
    
    print("Exported COLMAP DB and pairs.txt at :-")
    print(colmap_db_path)
    print(colmap_pairs_path)
    print("--"*50)

    return colmap_db_path, colmap_pairs_path



Writing mast3r_colmapdb_module.py


In [7]:
%%writefile imc25_prediction_utils.py

import json
import dataclasses
import numpy as np
import pandas as pd

@dataclasses.dataclass
class Prediction:
    image_id: str | None  # A unique identifier for the row -- unused otherwise. Used only on the hidden test set.
    dataset: str
    filename: str
    cluster_index: int | None = None
    rotation: np.ndarray | None = None
    translation: np.ndarray | None = None

def save_predictions(predictions, filepath):
    def convert(obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if dataclasses.is_dataclass(obj):
            return dataclasses.asdict(obj)
        raise TypeError(f"Type {type(obj)} not serializable")
    
    with open(filepath, "w") as f:
        json.dump(predictions, f, default=convert, indent=2)


def load_predictions(filepath):
    with open(filepath, "r") as f:
        data = json.load(f)
    
    return [
        Prediction(
            image_id=item["image_id"],
            dataset=item["dataset"],
            filename=item["filename"],
            cluster_index=item.get("cluster_index"),
            rotation=np.array(item["rotation"]) if item.get("rotation") is not None else None,
            translation=np.array(item["translation"]) if item.get("translation") is not None else None
        )
        for item in data
    ]



def write_to_submission_file(submission_file, dataset_preds, is_train):
    array_to_str = lambda array: ';'.join([f"{x:.09f}" for x in array])
    none_to_str = lambda n: ';'.join(['nan'] * n)
    
    with open(submission_file, 'w') as f:
        if is_train:
            f.write('dataset,scene,image,rotation_matrix,translation_vector\n')
            for dataset in dataset_preds:
                for prediction in dataset_preds[dataset]:
                    cluster_name = 'outliers' if prediction.cluster_index is None else f'cluster{prediction.cluster_index}'
                    rotation = none_to_str(9) if prediction.rotation is None else array_to_str(prediction.rotation.flatten())
                    translation = none_to_str(3) if prediction.translation is None else array_to_str(prediction.translation)
                    f.write(f'{prediction.dataset},{cluster_name},{prediction.filename},{rotation},{translation}\n')
        else:
            f.write('image_id,dataset,scene,image,rotation_matrix,translation_vector\n')
            for dataset in dataset_preds:
                for prediction in dataset_preds[dataset]:
                    cluster_name = 'outliers' if prediction.cluster_index is None else f'cluster{prediction.cluster_index}'
                    rotation = none_to_str(9) if prediction.rotation is None else array_to_str(prediction.rotation.flatten())
                    translation = none_to_str(3) if prediction.translation is None else array_to_str(prediction.translation)
                    f.write(f'{prediction.image_id},{prediction.dataset},{cluster_name},{prediction.filename},{rotation},{translation}\n')
    
    print("Submission File written to : ", submission_file)



def read_sample_submission_file(sample_submission_csv, is_train=False):
    samples = {}
    competition_data = pd.read_csv(sample_submission_csv)
    for _, row in competition_data.iterrows():
        # Note: For the test data, the "scene" column has no meaning, and the rotation_matrix and translation_vector columns are random.
        if row.dataset not in samples:
            samples[row.dataset] = []
        samples[row.dataset].append(
            Prediction(
                image_id=None if is_train else row.image_id,
                dataset=row.dataset,
                filename=row.image
            )
        )
    
    return samples



Writing imc25_prediction_utils.py


In [8]:
%%writefile imc25_mast3r_shortlist_matching_pycolmap_subp.py

import os
PREFIX_KAGGLE_ROOT = os.environ.get('PREFIX_KAGGLE_ROOT')
if PREFIX_KAGGLE_ROOT is None:
    raise "Undefined PREFIX_KAGGLE_ROOT in ENV vars !!!"

import argparse
from glob import glob
import shutil
import json
import dataclasses

import numpy as np
import torch

from copy import deepcopy
from time import sleep

import gc
import sys
sys.path.append(f'{PREFIX_KAGGLE_ROOT}/kaggle/working/mast3r/')
from retrieval_processor import get_retriever_model, get_similarity_pairs
from mast3r_colmapdb_module import get_colmapdb_from_mast3r_matching
from imc25_prediction_utils import Prediction, save_predictions, load_predictions, write_to_submission_file
from imc25_subp_scheduler import SubProcessScheduler


def get_image_pairs_shortlist_mast3r_retrieval(fnames, top_k=30, topk_percentile=0.3, top_k_max=50, 
                                               sim_score_thres=0.0, include_symmetric=False,
                                               device=torch.device('cpu')):

    retriever = get_retriever_model(device=device)
    sim_matrix = retriever(fnames)

    pairs = get_similarity_pairs(sim_matrix,
                top_k=top_k,
                topk_percentile=topk_percentile,
                top_k_max = top_k_max,
                sim_score_thres=sim_score_thres,
                include_symmetric=include_symmetric,
                )

    return pairs




if __name__ == "__main__":

    parser = argparse.ArgumentParser(description="MASt3R Shortlist and Matching SubP")

    parser.add_argument("--device", type=str, default="cuda" if torch.cuda.is_available() else "cpu")
    parser.add_argument("--data_dir", type=str, required=True, help="Root Dir of all datasets")
    parser.add_argument("--io_dir", type=str, required=True, help="input output dir")
    parser.add_argument("--dataset_split", type=str, default="test", choices=["train", "test"], help="input output dir")
    parser.add_argument("--max_images", type=int, default=None, help="max images to load for debugging")

    args = parser.parse_args()

    dataset_samples = load_predictions(os.path.join(args.io_dir, "dataset_samples.json"))
    dataset_filename_to_index = {p.filename: idx for idx, p in enumerate(dataset_samples)}
    dataset = dataset_samples[0].dataset
    is_train = (args.dataset_split == "train")

    images_dir = os.path.join(args.data_dir, args.dataset_split, dataset)
    images = [os.path.join(images_dir, sample.filename) for sample in dataset_samples]
    if args.max_images is not None:
        images = images[:args.max_images]

    index_pairs = get_image_pairs_shortlist_mast3r_retrieval(
        images,
        device=args.device,
        top_k=10, topk_percentile=0.3, top_k_max=30,
        sim_score_thres=0.001, include_symmetric=False,
    )

    database_path, pairs_txt_path = get_colmapdb_from_mast3r_matching(outputdir=args.io_dir, filelist=images, pair_idxs_list=index_pairs,
                                        shared_intrinsics = False, image_square_ok=True, device=args.device)


    workdir = os.path.dirname(args.io_dir)
    conda_mamba_path = "/kaggle/working/miniconda/bin/conda"

    pycolmap_subp_scheduler = SubProcessScheduler(workdir=workdir, env_dict={"PREFIX_KAGGLE_ROOT": PREFIX_KAGGLE_ROOT},
        base_cmd=f"{conda_mamba_path} run -n mast3r_sfm python imc25_mast3r_pycolmap_subp.py --data_dir={args.data_dir} --dataset_split={'train' if is_train else 'test'} ".split(),
        datasets_to_process=[dataset], process_type="pycolmap", num_devices=1, num_subprocesses_per_device=1, max_retries=10, max_duration=10*3600, device_type="cpu",
        )
    
    pycolmap_subp_scheduler.run()

    


Writing imc25_mast3r_shortlist_matching_pycolmap_subp.py


In [9]:
%%writefile imc25_mast3r_pycolmap_subp.py

import os
PREFIX_KAGGLE_ROOT = os.environ.get('PREFIX_KAGGLE_ROOT')
if PREFIX_KAGGLE_ROOT is None:
    raise "Undefined PREFIX_KAGGLE_ROOT in ENV vars !!!"

import argparse
from glob import glob
import shutil
import json
import dataclasses

import numpy as np
import pycolmap

from copy import deepcopy
from time import sleep
import gc

from imc25_prediction_utils import Prediction, save_predictions, load_predictions, write_to_submission_file


if __name__ == "__main__":

    parser = argparse.ArgumentParser(description="pycolmap subp")

    parser.add_argument("--data_dir", type=str, required=True, help="Root Dir of all datasets")
    parser.add_argument("--io_dir", type=str, required=True, help="input output dir")
    parser.add_argument("--dataset_split", type=str, default="test", choices=["train", "test"], help="input output dir")
    parser.add_argument("--max_images", type=int, default=None, help="max images to load for debugging")

    args = parser.parse_args()

    dataset_samples = load_predictions(os.path.join(args.io_dir, "dataset_samples.json"))
    dataset_filename_to_index = {p.filename: idx for idx, p in enumerate(dataset_samples)}
    dataset = dataset_samples[0].dataset

    images_dir = os.path.join(args.data_dir, args.dataset_split, dataset)
    images = [os.path.join(images_dir, sample.filename) for sample in dataset_samples]
    if args.max_images is not None:
        images = images[:args.max_images]

    database_path = os.path.join(args.io_dir, "colmap.db")
    pairs_txt_path = os.path.join(args.io_dir, "pairs.txt")

    matches_verified_indicator_file = os.path.join(args.io_dir, "matches_verified.txt")
    if not os.path.exists(matches_verified_indicator_file):
        pycolmap.verify_matches(database_path, pairs_txt_path)

    os.system(f'touch {matches_verified_indicator_file}')

    output_path = os.path.join(args.io_dir, 'colmap_rec_mast3r')
    if os.path.exists(output_path):
        shutil.rmtree(output_path)
    os.makedirs(output_path)


    # By default colmap does not generate a reconstruction if less than 10 images are registered.
    # Lower it to 3.
    mapper_options = pycolmap.IncrementalPipelineOptions()
    mapper_options.min_model_size = 3
    mapper_options.max_num_models = 25
    # mapper_options.num_threads = 1

    maps = pycolmap.incremental_mapping(
        database_path=database_path,
        image_path=images_dir,
        output_path=output_path,
        options=mapper_options)
    
    sleep(1)
    print(maps)


    registered = 0
    for map_index, cur_map in maps.items():
        for index, image in cur_map.images.items():
            prediction_index = dataset_filename_to_index[image.name]
            dataset_samples[prediction_index].cluster_index = map_index
            dataset_samples[prediction_index].rotation = deepcopy(image.cam_from_world.rotation.matrix())
            dataset_samples[prediction_index].translation = deepcopy(image.cam_from_world.translation)
            registered += 1
    

    mapping_result_str = f'Dataset "{dataset}" -> Registered {registered} / {len(images)} images with {len(maps)} clusters'
    print(mapping_result_str)

    gc.collect()

    save_predictions(dataset_samples, os.path.join(args.io_dir, "dataset_preds.json"))

    

Writing imc25_mast3r_pycolmap_subp.py


In [10]:
%%writefile imc25_subp_scheduler.py

import os
import time
import subprocess
from typing import List, Optional
from imc25_prediction_utils import Prediction, save_predictions, load_predictions, write_to_submission_file


class SubProcessScheduler:
    
    def __init__(
        self,
        workdir: str,
        env_dict: dict,
        base_cmd: List[str],
        datasets_to_process: List[str],
        process_type: str, # matching or pycolmap or etc 
        num_devices: int = 1,
        num_subprocesses_per_device: int = 1,
        max_retries: int = 3,
        max_duration: int = 3600,
        device_type: str = "cpu",
        dataset_samples: Optional[dict] = None,
        dataset_preds: Optional[dict] = None,
        is_train: bool = False,
    ):
        
        self.workdir = workdir
        self.env_dict = env_dict
        self.base_cmd = base_cmd
        self.datasets_to_process = datasets_to_process
        self.num_devices = num_devices
        self.num_subprocesses_per_device = num_subprocesses_per_device
        self.max_retries = max_retries
        self.max_duration = max_duration
        self.device_type = device_type
        self.process_type = process_type
        self.dataset_samples = dataset_samples
        self.dataset_preds = dataset_preds
        self.is_train = is_train
        
        if device_type == "cpu":
            assert self.num_devices == 1
        elif device_type == "cuda":  # hardcoded requirements for now
            assert max_retries == 1
            assert dataset_samples is not None
        else:
            raise "Unknown Device Type !!!"

        # Internal state
        self._pending_datasets: List[str] = []
        self._retry_queue: dict[str, int] = {}  # dataset_name -> retry_count
        self._attempted_datasets: set[str] = set()
        self._final_failed_datasets: List[str] = []
        self._start_time: Optional[float] = None
        self._initialize_pending_list()

    def _initialize_pending_list(self):
        self._pending_datasets = self.datasets_to_process.copy()


    def launch_subprocess(self, dataset: str, process_type: str):
        
        io_dir = os.path.join(self.workdir, dataset)
        os.makedirs(io_dir, exist_ok=True)

        print(f"Launching {process_type} subprocess for dataset: '{dataset}', io_dir='{io_dir}'")

        subprocess_env = os.environ.copy()
        for env_var_key, env_var_val in self.env_dict.items():
            subprocess_env[env_var_key] = env_var_val

        # for now: cuda --> mast3r subp ; cpu --> pycolmap subp
        if self.device_type == "cuda":
            # hardcoded the requirements for mast3r matching subprocess here, later needs to be decoupled
            save_predictions(self.dataset_samples[dataset], os.path.join(io_dir, "dataset_samples.json"))
            device_for_curr_dataset = f"{self.datasets_to_process.index(dataset) % self.num_devices}"
            subprocess_env["CUDA_VISIBLE_DEVICES"] = device_for_curr_dataset            
            cmd = self.base_cmd + [f"--io_dir={io_dir}", f"--device=cuda:0"]

        else:
            cmd = self.base_cmd + [f"--io_dir={io_dir}"]

        stdout_path = os.path.join(io_dir, f"proc_{dataset}_{process_type}_log.txt")
        stdout_file = open(stdout_path, "w")

        p = subprocess.Popen(
            cmd,
            env=subprocess_env,
            stdout=stdout_file,
            stderr=stdout_file,
        )

        return p, stdout_file

    def run(self, external_start_time=None, submission_file=None):
        
        if self._start_time is None:
            if external_start_time is None:
                self._start_time = time.time()
            else:
                self._start_time = external_start_time
        else:
            # If run() is called more than once, reset internal state.
            self._start_time = time.time()
            self._retry_queue.clear()
            self._attempted_datasets.clear()
            self._final_failed_datasets.clear()
            self._initialize_pending_list()

        batch_size = self.num_devices * self.num_subprocesses_per_device

        while self._pending_datasets or self._retry_queue:
            subprocesses = []

            while self._pending_datasets and len(subprocesses) < batch_size:
                dataset = self._pending_datasets.pop(0)
                self._attempted_datasets.add(dataset)

                p, out_f = self.launch_subprocess(dataset, self.process_type)
                subprocesses.append((dataset, p, out_f))

            for dataset, p, out_f in subprocesses:
                p.wait()
                out_f.close()

                if p.returncode != 0:
                    print(f"[ERROR] {self.process_type} Subprocess for '{dataset}' failed with returncode={p.returncode}")
                    current_retry_count = self._retry_queue.get(dataset, 0) + 1
                    if current_retry_count < self.max_retries:
                        self._retry_queue[dataset] = current_retry_count
                        print(f"  → Scheduling retry #{current_retry_count} for '{dataset}'")
                    else:
                        print(f"[FAILURE] {self.process_type} Subprocess for '{dataset}' failed after {current_retry_count} attempts.")
                        self._final_failed_datasets.append(dataset)
                        self._retry_queue.pop(dataset, None)
                else:
                    self._retry_queue.pop(dataset, None)
                    print(f"[SUCCESS] {self.process_type} Subprocess for '{dataset}' processed successfully.")
                    dataset_pred_json_file = os.path.join(self.workdir, dataset, "dataset_preds.json")
                    if os.path.exists(dataset_pred_json_file) and self.dataset_preds is not None:
                        self.dataset_preds[dataset] = load_predictions(dataset_pred_json_file)
                        if submission_file is not None:
                            write_to_submission_file(submission_file, self.dataset_preds, self.is_train)
            
            retrying_now = [d for d in self._retry_queue.keys() if d not in self._pending_datasets]
            for d in retrying_now:
                self._pending_datasets.append(d)

            elapsed = time.time() - self._start_time
            all_tried_once = len(self._attempted_datasets) >= len(self.datasets_to_process)

            if elapsed > self.max_duration and all_tried_once:
                print(
                    f"\n Time limit of {self.max_duration}s reached "
                    f"and all datasets attempted at least once."
                )
                break

        if self._final_failed_datasets:
            print("\nDatasets that failed after maximum retries:")
            for ds in self._final_failed_datasets:
                print(f"  - {ds}")
        else:
            print("\nAll datasets processed successfully.")



Writing imc25_subp_scheduler.py


In [11]:
import sys
import os
import time
import gc
import numpy as np
import dataclasses
import pandas as pd
import shutil
import json
import subprocess
from typing import List, Optional
from copy import deepcopy

from imc25_prediction_utils import Prediction, save_predictions, load_predictions, write_to_submission_file, read_sample_submission_file
from imc25_subp_scheduler import SubProcessScheduler


In [12]:
test_dataset_dir = f'{PREFIX_KAGGLE_ROOT}/kaggle/input/image-matching-challenge-2025/test/'
is_competition = (len(os.listdir(test_dataset_dir)) > 2)
print(f"{is_competition=}")

is_competition=False


In [13]:
is_train = False
max_images = None  # Used For debugging only. Set to None to disable.
# max_images = 20

data_dir = f'{PREFIX_KAGGLE_ROOT}/kaggle/input/image-matching-challenge-2025'
workdir = f'{PREFIX_KAGGLE_ROOT}/kaggle/working/result/'

if os.path.exists(workdir):
    shutil.rmtree(workdir)
os.makedirs(workdir)

if is_train:
    sample_submission_csv = os.path.join(data_dir, 'train_labels.csv')
else:
    sample_submission_csv = os.path.join(data_dir, 'sample_submission.csv')

samples = read_sample_submission_file(sample_submission_csv, is_train)

for dataset in samples:
    print(f'Dataset "{dataset}" -> num_images={len(samples[dataset])}')


gc.collect()

if is_train:
    # max_images = 5

    # Note: When running on the training dataset, the notebook will hit the time limit and die. Use this filter to run on a few specific datasets.
    datasets_to_process = [
    	# New data.
    	# 'amy_gardens',
    	'ETs',
    	# 'fbk_vineyard',
    	'stairs',
    	# Data from IMC 2023 and 2024.
    	# 'imc2024_dioscuri_baalshamin',
    	# 'imc2023_theather_imc2024_church',
    	# 'imc2023_heritage',
    	# 'imc2023_haiper',
    	# 'imc2024_lizard_pond',
    	# Crowdsourced PhotoTourism data.
    	# 'pt_stpeters_stpauls',
    	# 'pt_brandenburg_british_buckingham',
    	# 'pt_piazzasanmarco_grandplace',
    	# 'pt_sacrecoeur_trevi_tajmahal',
    ]

else:
    datasets_to_process = [dataset for dataset in samples]


print()
# sort datasets acc to length
datasets_to_process = sorted(datasets_to_process, key=lambda dataset: len(samples[dataset]))
# for debug purposes
if not is_competition:
    datasets_to_process = [datasets_to_process[0]]
    

print(f"{datasets_to_process=}")
print("--"*30)

dataset_preds = deepcopy(samples)



Dataset "ETs" -> num_images=22
Dataset "amy_gardens" -> num_images=200
Dataset "fbk_vineyard" -> num_images=163
Dataset "imc2023_haiper" -> num_images=54
Dataset "imc2023_heritage" -> num_images=209
Dataset "imc2023_theather_imc2024_church" -> num_images=76
Dataset "imc2024_dioscuri_baalshamin" -> num_images=138
Dataset "imc2024_lizard_pond" -> num_images=214
Dataset "pt_brandenburg_british_buckingham" -> num_images=225
Dataset "pt_piazzasanmarco_grandplace" -> num_images=168
Dataset "pt_sacrecoeur_trevi_tajmahal" -> num_images=225
Dataset "pt_stpeters_stpauls" -> num_images=200
Dataset "stairs" -> num_images=51

datasets_to_process=['ETs']
------------------------------------------------------------


In [14]:
conda_mamba_path = "/kaggle/working/miniconda/bin/conda"

mast3r_matching_subp_scheduler = SubProcessScheduler(workdir=workdir, env_dict={"PREFIX_KAGGLE_ROOT": PREFIX_KAGGLE_ROOT, "MPLBACKEND": "tkagg"},
     base_cmd=f"{conda_mamba_path} run -n mast3r_sfm python imc25_mast3r_shortlist_matching_pycolmap_subp.py --data_dir={data_dir} --dataset_split={'train' if is_train else 'test'} ".split(),
     datasets_to_process=datasets_to_process, dataset_samples=samples, dataset_preds=dataset_preds, process_type="matching",
     num_devices=2, num_subprocesses_per_device=1, max_retries=1, max_duration=10*3600, device_type="cuda",
     )

mast3r_matching_subp_scheduler.run(submission_file=SUBMISSION_FILE)



Launching matching subprocess for dataset: 'ETs', io_dir='/kaggle/working/result/ETs'
[SUCCESS] matching Subprocess for 'ETs' processed successfully.
Submission File written to :  /kaggle/working/submission.csv

All datasets processed successfully.


In [15]:
# collect dataset_preds.json from each dataset io_dir

dataset_preds = deepcopy(samples)

for dataset in dataset_preds:
    dataset_pred_json_file = os.path.join(workdir, dataset, "dataset_preds.json")
    if os.path.exists(dataset_pred_json_file):
        dataset_preds[dataset] = load_predictions(dataset_pred_json_file)

write_to_submission_file(SUBMISSION_FILE, dataset_preds, is_train)


Submission File written to :  /kaggle/working/submission.csv


In [16]:
# for debug purposes
if not is_competition and not is_train:
    print("--"*40)
    print("[DEBUG]")
    if os.path.exists('/kaggle/working/result/ETs/proc_ETs_pycolmap_log.txt'):
        !cat /kaggle/working/result/ETs/proc_ETs_pycolmap_log.txt
    else:
        print("ETs pycolmap log not found !!!")
    print("--"*40)



--------------------------------------------------------------------------------
[DEBUG]
I20250617 18:23:19.465921 131932378625600 misc.cc:44] 
Feature matching
I20250617 18:23:19.466665 131932370232896 sift.cc:1432] Creating SIFT CPU feature matcher
I20250617 18:23:19.466773 131932361840192 sift.cc:1432] Creating SIFT CPU feature matcher
I20250617 18:23:19.466950 131932345054784 sift.cc:1432] Creating SIFT CPU feature matcher
I20250617 18:23:19.467021 131932353447488 sift.cc:1432] Creating SIFT CPU feature matcher
I20250617 18:23:19.467069 131932378625600 pairing.cc:742] Importing image pairs...
I20250617 18:23:19.467320 131932378625600 pairing.cc:775] Matching block [1/1]
I20250617 18:23:21.294299 131932378625600 feature_matching.cc:46] in 1.827s
I20250617 18:23:21.299074 131932378625600 timer.cc:91] Elapsed time: 0.031 [minutes]
I20250617 18:23:21.356667 131932658775872 incremental_pipeline.cc:237] Loading database
I20250617 18:23:21.358011 131932658775872 database_cache.cc:66] Load

In [17]:
%cd /kaggle/working/
!rm -r /kaggle/working/mast3r/
!rm -r /kaggle/working/miniconda/
!rm -r /kaggle/working/result/

/kaggle/working
